In [56]:
import cv2
import numpy as np
import serial
#import time
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [57]:
cam = 2
fps = 240

model_version = "my_model_13.0.keras"
Im_Width = 128
Im_Height = 128

In [58]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [59]:
def measure_brightness(frame):
    cropped_frame = crop_image(frame, 120, 0, 400, 250)
    gray = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [60]:
def select_best_frame(frame_buffer):
    best_frame = None
    best_score = -1 

    for frame in frame_buffer:
        score = measure_brightness(frame)
        if score > best_score:
            best_score = score
            best_frame = frame

    return best_frame


In [61]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [62]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    predictions = model.predict(frame_array)  # Outputs probabilities for each class
    predicted_class = np.argmax(predictions[0])  # Get the index of the class with the highest probability
    confidence = predictions[0][predicted_class]  # Confidence score for the predicted class

    # Map class indices to labels
    class_labels = {0: "Defective", 2: "Non-Defective", 1: "No Bottle"}
    predicted_label = class_labels[predicted_class]

    send_to_arduino(arduino, predicted_class)
    
    return predicted_label, confidence

In [63]:
def capture(cam, fps, model_path, target_size=(Im_Width, Im_Height)):
    cap = cv2.VideoCapture(cam)  # Open webcam
    # Set desired frame rate
    desired_fps = fps
    cap.set(cv2.CAP_PROP_FPS, desired_fps)

    frame_buffer = []
    max_frames = 60  # Number of frames to evaluate

    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:

        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            feedback, confidence = predict_frame(best_frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(best_frame, f"Confidence: {confidence:.2f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Display the best frame
            cv2.imshow("Best Frame", best_frame)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [64]:
def setup_serial_connection(port='COM4', baud_rate=9600):
    arduino = serial.Serial(port, baud_rate, timeout=1)
    print("Arduino connected. Waiting for initialization...")
    arduino.flush()  # Clear any remaining data
    return arduino

In [65]:
def send_to_arduino(arduino, value):
    arduino.write(f"{value}\n".encode())
    print(f"Sent: {value}")


In [66]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    arduino = setup_serial_connection('COM4')  # Connect once
    capture(cam, fps, model_path, target_size=(Im_Width, Im_Height))

Arduino connected. Waiting for initialization...
Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.89)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.94)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.95)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.89)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.93)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.96)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.91)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.91)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.89)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Sent: 1
Prediction: No Bottle (Confidence: 0.91)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/ste